# The Toy Shape Dataset

 - A toy shapes dataset: small resolution image composed of squares, circles an triangles.
 - Strongly inspired from [this implementation](https://github.com/matterport/Mask_RCNN/blob/cbff80f3e3f653a9eeee43d0d383a0385aba546b/samples/shapes/shapes.py).
 - The [config.yaml file](./config.yaml) should be able to train the dataset composed of 50 images in less than 10 minutes on a regular GPU with a **mAP@[0.5:0.95] of ~70%**.
 
This dataset is ideal for debugging and educational purpose..

In [39]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tqdm

import sys; sys.path.append("../../../")
import maskflow

import utils

root_dir = Path("/home/hadim/.data/Neural_Network/Maskflow/Shapes")

data_dir = root_dir / "Data"
data_dir.mkdir(parents=True, exist_ok=True)

# Load the configuration.
config = maskflow.load_config("config.yaml")

# Copy config next to data folder.
maskflow.save_config(config, root_dir / "config.yaml")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generate the dataset

In [2]:
height = config["DATASET"]["IMAGE_SIZE"]
width = config["DATASET"]["IMAGE_SIZE"]
class_names = config["DATASET"]["CLASS_NAMES"]

count = 50
min_n_per_image = 1
max_n_per_image = 4
training_size = 0.9  # From 0 to 1

image_format = "png"
masks_format = "png"

train_ids, _ = train_test_split(np.arange(0, count), train_size=training_size)

train_features_example = []
test_features_example = []

train_file_path = data_dir / "train.tfrecords"
test_file_path = data_dir / "test.tfrecords"

for image_id in tqdm.trange(count):
    # Generate image specification
    bg_color, shapes = utils.random_image(height, width, min_n_per_image, max_n_per_image, class_names)
    
    # Generate the image
    image = utils.generate_image(bg_color, height, width, shapes)
    
    # Generate the masks
    masks, label_ids = utils.generate_mask(bg_color, height, width, shapes, class_names)
    
    # Get a list of class from the object label ids.
    label_names = [class_names[class_id].encode("utf8") for class_id in label_ids]
    
    # Get bounding boxes from masks.
    bboxes = maskflow.bbox.from_masks(masks)
    
    filename = f"toy_{image_id:04d}.png"
    
    build_features_args = {}
    build_features_args['image'] = image
    build_features_args['image_id'] = image_id
    build_features_args['filename'] = filename
    build_features_args['image_format'] = image_format
    build_features_args['bboxes'] = bboxes
    build_features_args['masks'] = masks
    build_features_args['label_ids'] = label_ids
    build_features_args['label_names'] = label_names
    build_features_args['masks_format'] = masks_format
    features_dict = maskflow.dataset.build_features_dict(**build_features_args)
    
    example = tf.train.Example(features=tf.train.Features(feature=features_dict))
    
    if image_id in train_ids:
        train_features_example.append(example)
    else:
        test_features_example.append(example)
        
# Write examples to TFRecord files.
with tf.io.TFRecordWriter(str(train_file_path)) as writer:
    for example in train_features_example:
        writer.write(example.SerializeToString())
        
with tf.io.TFRecordWriter(str(test_file_path)) as writer:
    for example in test_features_example:
        writer.write(example.SerializeToString())

100%|██████████| 50/50 [00:04<00:00, 11.00it/s]


## Visualize some images with their masks

### TODO

- allow iamge without bboxes
- convert to bboxes
- pad datum to be able to batch
- make plot

In [56]:
train_dataset = maskflow.dataset.parse(train_file_path, config, do_preprocess=True)

for feature in train_dataset:
    pass

In [74]:
for feature in train_dataset.batch(10):
    print("p")
feature['masks'].shape

p
p
p
p
p


TensorShape([5, 20, 128, 128])

In [84]:
dataset = train_dataset.shuffle(1000).take(10)
features = [feature for feature in dataset]

In [85]:
feature = features[0]

image = feature['image'].numpy()
masks = feature['masks'].numpy()
bboxes = feature['bboxes'].numpy()
label_ids = feature['label_ids'].numpy()
label_names = feature['label_names'].numpy()

In [ ]:
def batch_display_top_masks(batch_image, batch_target, batch_idx, categories,
                            basesize=14, limit=4, cmap="PuBu_r",
                            pixel_mean=[0, 0, 0], pixel_std=[1, 1, 1]):
    """Display the given images and the top few class masks.

    See `maskflow.display_top_masks`.
    """
    for image, target, idx in zip(batch_image.tensors, batch_target, batch_idx):
        image = image.numpy().copy().swapaxes(0, 2).swapaxes(0, 1)

        # Undo image normalization
        image += np.array(pixel_mean)
        image *= np.array(pixel_std)
        image = image.astype('uint8')

        labels = target.get_field('labels')
        masks = target.get_field('masks')

        display_top_masks(image, masks, labels, categories, basesize=basesize, limit=limit, cmap=cmap)

In [ ]:
# Number of batch to load
n = 1

# Load some data
data_loader = maskflow.dataset.get_data_loader(config, data_dir, is_train=True)
some_data = [iter(data_loader).next() for _ in range(n)]

# Retrieve category's names
categories = data_loader.dataset.coco.cats

for batch_image, batch_target, batch_idx in some_data:
    maskflow.viz.batch_display_top_masks(train_dataset, basesize=14, limit=3, cmap="PuBu_r")